<a href="https://colab.research.google.com/github/Tessellate-Imaging/Monk_Object_Detection/blob/master/application_model_zoo/Example%20-%20logo%20detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents


## 1. Installation Instructions



## 2. Use trained model to segment roads in satellite imagery



## 3. How to train a custom segmenter using "Massachusetts Roads Dataset"

# About the networks


1. UNet
  - https://arxiv.org/abs/1505.04597
  - https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47
  - https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5

2. FPN
  - http://openaccess.thecvf.com/content_cvpr_2017/papers/Lin_Feature_Pyramid_Networks_CVPR_2017_paper.pdf
  - https://towardsdatascience.com/review-fpn-feature-pyramid-network-object-detection-262fc7482610
  - https://medium.com/@jonathan_hui/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c


3. PSPNet
  - https://arxiv.org/abs/1612.01105
  - https://towardsdatascience.com/review-pspnet-winner-in-ilsvrc-2016-semantic-segmentation-scene-parsing-e089e5df177d
  - https://developers.arcgis.com/python/guide/how-pspnet-works/


4. Linknet
  - https://arxiv.org/pdf/1707.03718.pdf
  - https://neptune.ai/blog/image-segmentation-tips-and-tricks-from-kaggle-competitions

# Installation

 - Run these commands
     
     - git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git
     
     - cd Monk_Object_Detection/9_segmentation_models/installation
     
 - Select the right requirements file and run
 
     - cat requirements_cuda9.0.txt | xargs -n 1 -L 1 pip install

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
# For colab use the command below
! cd Monk_Object_Detection/3_mxrcnn/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install


# For Local systems and cloud select the right CUDA version
#! cd Monk_Object_Detection/3_mxrcnn/installation && cat requirements_cuda10.0.txt | xargs -n 1 -L 1 pip install

# Use already trained model for demo

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from infer_segmentation import Infer

In [ ]:
gtf = Infer();

In [ ]:
classes_dict = {
    'background': 0,
    'road': 1
};
classes_to_train = ['road'];

In [ ]:
gtf.Data_Params(classes_dict, classes_to_train, image_shape=[1500, 1500])

In [ ]:
# Download trained model

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1xGBGpttQs1XxXJJnP9HSRo3yCZH4dj4R' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1xGBGpttQs1XxXJJnP9HSRo3yCZH4dj4R" -O seg_road_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq seg_road_trained.zip

In [ ]:
gtf.Model_Params(model="Unet", backbone="efficientnetb3", path_to_model='seg_road_trained/best_model.h5')

In [ ]:
gtf.Setup();

In [ ]:
gtf.Predict("seg_road_trained/test/img-1.png", vis=True);

In [ ]:
gtf.Predict("seg_road_trained/test/img-2.png", vis=True);

In [ ]:
gtf.Predict("seg_road_trained/test/img-3.png", vis=True);

In [ ]:
gtf.Predict("seg_road_trained/test/img-6.png", vis=True);

In [ ]:
gtf.Predict("seg_road_trained/test/img-7.png", vis=True);

In [ ]:
gtf.Predict("seg_road_trained/test/img-10.png", vis=True);

# Train you own detector

# Monk Format

## Dataset Directory Structure

      root_dir
          |
          | 
          |         
          |----train_img_dir
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on) 
          |
          |----train_mask_dir
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)
          |
          |----val_img_dir (optional)
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)
          |
          |----val_mask_dir (optional)
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)

          
          

# Sample Dataset Credits

    credits: https://www.kaggle.com/insaff/massachusetts-roads-dataset

In [ ]:
! pip install kaggle

In [ ]:
! kaggle datasets download insaff/massachusetts-roads-dataset

In [ ]:
! unzip -qq massachusetts-roads-dataset.zip -d road_dataset

In [ ]:
ls road_dataset/road_segmentation_ideal/training/

In [ ]:
os.mkdir("road_dataset/road_segmentation_ideal/training/output_updated")

In [ ]:
os.mkdir("road_dataset/road_segmentation_ideal/training/input_updated")

In [ ]:
import cv2
import os
import numpy as np
from tqdm.notebook import tqdm

lst = os.listdir("road_dataset/road_segmentation_ideal/training/output")

for i in tqdm(range(len(lst))):
    img_name = lst[i];
    img = cv2.imread("road_dataset/road_segmentation_ideal/training/output/" + lst[i], 0);
    img[img==255]=1;
    img = np.pad(a, 1, mode='constant')
    cv2.imwrite("road_dataset/road_segmentation_ideal/training/output_updated/" + lst[i], img);
    
    
    img_name = lst[i];
    img = cv2.imread("road_dataset/road_segmentation_ideal/training/output/" + lst[i], 0);
    img[img==255]=1;
    cv2.imwrite("road_dataset/road_segmentation_ideal/training/output_updated/" + lst[i], img);
    
    os.system("cp road_dataset/road_segmentation_ideal/training/input/" + lst[i] 
              + " road_dataset/road_segmentation_ideal/training/input_updated/")

In [ ]:
import os
print(len(os.listdir("road_dataset/road_segmentation_ideal/training/output/")))
print(len(os.listdir("road_dataset/road_segmentation_ideal/training/output_updated/")))
print(len(os.listdir("road_dataset/road_segmentation_ideal/training/input/")))
print(len(os.listdir("road_dataset/road_segmentation_ideal/training/input_updated/")))

## Training 

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from train_segmentation import Segmenter

In [ ]:
gtf = Segmenter();

In [ ]:
img_dir = "road_dataset/road_segmentation_ideal/training/input_updated";
mask_dir = "road_dataset/road_segmentation_ideal/training/output_updated";

In [ ]:
classes_dict = {
    'background': 0,
    'road': 1
};
classes_to_train = ['road'];

In [ ]:
gtf.Train_Dataset(img_dir, mask_dir, classes_dict, classes_to_train)

In [ ]:
gtf.Val_Dataset(img_dir, mask_dir)

In [ ]:
gtf.List_Backbones();

In [ ]:
gtf.Data_Params(batch_size=2, backbone="efficientnetb3", image_shape=[1500, 1500])

In [ ]:
gtf.List_Models();

In [ ]:
gtf.Model_Params(model="Unet")

In [ ]:
gtf.Train_Params(lr=0.0001)

In [ ]:
gtf.Setup();

In [ ]:
gtf.Train(num_epochs=2);

In [ ]:
gtf.Visualize_Training_History();

# Inference

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from infer_segmentation import Infer

In [ ]:
gtf = Infer();

In [ ]:
classes_dict = {
    'background': 0,
    'road': 1
};
classes_to_train = ['road'];

In [ ]:
gtf.Data_Params(classes_dict, classes_to_train, image_shape=[1500, 1500])

In [ ]:
gtf.Model_Params(model="Unet", backbone="efficientnetb3", path_to_model='best_model.h5')

In [ ]:
gtf.Setup();

In [ ]:
gtf.Predict("road_dataset/road_segmentation_ideal/training/input/img-271.png", vis=True);